# Assess & Clean Data

Load raw data, assess, clean and save in proper sets.


### Data Sources

- portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
- profile.json - demographic data for each customer
- transcript.json - records for transactions, offers received, offers viewed, and offers completed

### Changes

- 2018-12-19: Started project



In [1]:
# load libraries

import numpy as np
import pandas as pd
from tqdm import tqdm

# my own custom functions
import EDA_functions as EDA
import cleaning_functions as cleaning

# visualization
import matplotlib.pyplot as plt
import seaborn as sns #, sns.set_style('whitegrid')
color = 'rebeccapurple'
%matplotlib inline

# display settings
from IPython.display import display
pd.options.display.max_columns = None

from pathlib import Path  # to make file path references relative to notebook directory

In [2]:
# import data

portfolio_file = Path.cwd() / "data" / "raw" / "portfolio.json"
profile_file = Path.cwd() / "data" / "raw" / "profile.json"
transcript_file = Path.cwd() / "data" / "raw" / "transcript.json"

portfolio = pd.read_json(portfolio_file, orient='records', lines=True)
profile = pd.read_json(profile_file, orient='records', lines=True)
transcript = pd.read_json(transcript_file, orient='records', lines=True)

## Assess Data
### Check portfolio data

In [3]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
channels      10 non-null object
difficulty    10 non-null int64
duration      10 non-null int64
id            10 non-null object
offer_type    10 non-null object
reward        10 non-null int64
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


In [4]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


Explanations: 
- id (string) - offer id
- offer_type (string) - type of offer ie BOGO (buy-one-get-one-free), discount, informational
- difficulty (int) - minimum required spend to complete an offer
- reward (int) - reward given for completing an offer
- duration (int) - 
- channels (list of strings)

### Check profile data

In [5]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
age                 17000 non-null int64
became_member_on    17000 non-null int64
gender              14825 non-null object
id                  17000 non-null object
income              14825 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.1+ KB


In [6]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


Explanations:

* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

### Check transcript data

In [7]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
event     306534 non-null object
person    306534 non-null object
time      306534 non-null int64
value     306534 non-null object
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [8]:
transcript.sample(10)

,event,person,time,value
236276,offer viewed,5d6346117b314a87b94492d04fa2d71e,546,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
283696,transaction,eef5ea58662f4e27b591d39990c10ae2,624,{'amount': 23.48}
142891,offer viewed,7986640ceede4113872bcfa8c2bc69dc,378,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
299703,transaction,c1f57b2c8e13441bafc67d592db48dd3,684,{'amount': 3.35}
53566,offer received,91e8ec094d0a4e1ba88bf1b346c62762,168,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
191621,transaction,b66f9cb486024f0d87ab7469e8d7528d,468,{'amount': 7.11}
178940,transaction,b095142ced7748afb287c9634f2b794e,432,{'amount': 18.48}
253138,offer received,492a8df7d5e44c019fbb85992ac58862,576,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
61426,offer received,6cb26ad67f5e43ffb4f9b435d96f6615,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
40811,transaction,b6874ec74838469482e7bcce0251acdd,96,{'amount': 2.3}


Explanations:
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours after start of 30 days test period. The data begins at time t=0
* value (dict of strings) - either an offer id or transaction amount depending on the record

## Clean Data

Cleaning tasks:
- portfolio: rename `id` col to `offer_id`
- portfolio: one-hot-encode `channels`
- portfolio: add `prop_reward` (`income` / `difficulty`)
- portfolio add `rel_difficulty` (`difficulty` / `duration` in days)
- portfolio: add `duration_hours` col 
- profile: rename `id` col to `person_id`
- profile: change `became_member_on` to `member_since` with duration integer (days, starting from max date)
- transcript: rename `person` to `person_id`
- transcript: create two separate columns for offer ids and amounts in `values`
- all files: simplify ids

Further preparation tasks:
- transcript: flag every transaction if within valid period of a viewed promotion
- profile: add infos about consumption (number of purchases, amount per day etc.)
    - (could later be added for every promo)

In [9]:
"""rename columns"""

portfolio.rename(columns={'id': 'offer_id'}, inplace=True)
profile.rename(columns={'id' : 'person_id'}, inplace=True)
transcript.rename(columns={'person' : 'person_id'}, inplace=True)

In [10]:
# check results
for df in [portfolio, profile, transcript]:
    print(df.columns)

Index(['channels', 'difficulty', 'duration', 'offer_id', 'offer_type',
       'reward'],
      dtype='object')
Index(['age', 'became_member_on', 'gender', 'person_id', 'income'], dtype='object')
Index(['event', 'person_id', 'time', 'value'], dtype='object')


In [11]:
"""one-hot encode channels, set dtypes"""

for index, row in portfolio.iterrows():
    channel_string = ''.join(str(e) for e in portfolio.loc[index, 'channels'])
    for channel in ['web', 'email', 'mobile', 'social']:
        if channel in channel_string:
            portfolio.loc[index, channel] = 1
        else:
            portfolio.loc[index, channel] = 0
            
for col in portfolio[['web', 'email', 'mobile', 'social', 'offer_type']]:
    portfolio[col] = portfolio[col].astype('category', inplace=True)
portfolio.drop('channels', axis=1, inplace=True)

In [12]:
# check results
portfolio

,difficulty,duration,offer_id,offer_type,reward,web,email,mobile,social
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,0.0,1.0,1.0,1.0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1.0,1.0,1.0,1.0
2,0,4,3f207df678b143eea3cee63160fa8bed,informational,0,1.0,1.0,1.0,0.0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,1.0,1.0,1.0,0.0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,1.0,1.0,0.0,0.0
5,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3,1.0,1.0,1.0,1.0
6,10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2,1.0,1.0,1.0,1.0
7,0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0,0.0,1.0,1.0,1.0
8,5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,1.0,1.0,1.0,1.0
9,10,7,2906b810c7d4411798c6938adc9daaa5,discount,2,1.0,1.0,1.0,0.0


In [13]:
"""add prop_rewards, rel_difficulty and duration_hours"""

portfolio['prop_rewards'] = portfolio['reward'] / portfolio['difficulty']
portfolio['rel_difficulty'] = portfolio['difficulty'] / portfolio['duration']
portfolio['duration_hours'] = portfolio['duration'] * 24

In [14]:
# check results
display(portfolio.head())
display(portfolio.info())

,difficulty,duration,offer_id,offer_type,reward,web,email,mobile,social,prop_rewards,rel_difficulty,duration_hours
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,0.0,1.0,1.0,1.0,1.00,1.428571,168
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1.0,1.0,1.0,1.0,1.00,2.000000,120
2,0,4,3f207df678b143eea3cee63160fa8bed,informational,0,1.0,1.0,1.0,0.0,NaN,0.000000,96
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,1.0,1.0,1.0,0.0,1.00,0.714286,168
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,1.0,1.0,0.0,0.0,0.25,2.000000,240


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 12 columns):
difficulty        10 non-null int64
duration          10 non-null int64
offer_id          10 non-null object
offer_type        10 non-null category
reward            10 non-null int64
web               10 non-null category
email             10 non-null category
mobile            10 non-null category
social            10 non-null category
prop_rewards      8 non-null float64
rel_difficulty    10 non-null float64
duration_hours    10 non-null int64
dtypes: category(5), float64(2), int64(4), object(1)
memory usage: 1.1+ KB


None

In [15]:
"""change `became_member_on` to `member_since` with duration integer (days, starting from maxd ate)"""

profile['became_member_on'] = profile['became_member_on'].astype(str) \
    .astype('datetime64[ns]', format = "%Y%m%d")

max_date = profile['became_member_on'].max()
profile['member_since'] = profile['became_member_on'].apply(lambda x: (max_date - x).days)
profile.drop('became_member_on', axis=1, inplace=True)
print("max_date: ", max_date)

max_date:  2018-07-26 00:00:00


In [16]:
# check results
profile.head()

,age,gender,person_id,income,member_since
0,118,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,529
1,55,F,0610b486422d4921ae7d2bf64640c50b,112000.0,376
2,118,None,38fe809add3b4fcf9315a9694bb96ff5,NaN,14
3,75,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0,443
4,118,None,a03223e636434f42ac4c3df47e8bac43,NaN,356


In [17]:
"""create two separate columns for offer ids and amounts in transcript `values`"""

# see here why list call: https://stackoverflow.com/questions/17322668/typeerror-dict-keys-object-does-not-support-indexing
transcript['key'] = transcript['value'].apply(lambda x: list(x.keys())[0])
transcript['amount'] = transcript['value'].apply(lambda x: list(x.values())[0])
transcript['offer_id'] = transcript['amount']

transcript['amount'] = np.where(transcript['key'] == 'offer id', np.NaN, transcript['amount'])
transcript['amount'] = np.where(transcript['key'] == 'offer_id', np.NaN, transcript['amount'])
transcript['offer_id'] = np.where(transcript['key'] == 'amount', np.NaN, transcript['offer_id'])

transcript['amount'].astype(np.float64)
transcript.drop(['value', 'key'], axis=1, inplace=True)

In [18]:
# check result
transcript.sample(6)

,event,person_id,time,amount,offer_id
221837,transaction,bd3714abf1654f58939f379f88378882,510,3.51,NaN
24245,transaction,97b2476046874de3975e8feee95a1f98,30,2.75,NaN
212694,offer received,93afa7690b4545beaa80d6137191a204,504,NaN,2906b810c7d4411798c6938adc9daaa5
54416,offer received,120dee561e744f748747331688866adf,168,NaN,2906b810c7d4411798c6938adc9daaa5
266019,offer viewed,f8e6f303728449f0a610f7fce625d1da,588,NaN,5a8bc65990b245e5a138643cd4eb9837
182769,transaction,21f58f38b6ac4f67acd2959072adaa7c,444,8.25,NaN


In [19]:
"""simplify ids"""

def map_id(df, column, prefix=None):
    """Map IDs in a DataFrame column to encoded ones, return list of encoded IDs
    and a mapping dict that can be used to encode other dataframes."""
    
    id_dict = dict()
    count = 1
    encoded_ids = []
    
    for old_val in df[column]:
        if old_val not in id_dict:
            id_dict[old_val] = str(prefix) + str(count)
            count+=1
        
        encoded_ids.append(id_dict[old_val])
    
    return encoded_ids, id_dict

In [20]:
# encode person ids in profile
encoded_person_ids, person_id_dict = map_id(profile, 'person_id', prefix="p_")
profile['person_id'] = encoded_person_ids

#check results
profile.head()

,age,gender,person_id,income,member_since
0,118,None,p_1,NaN,529
1,55,F,p_2,112000.0,376
2,118,None,p_3,NaN,14
3,75,F,p_4,100000.0,443
4,118,None,p_5,NaN,356


In [21]:
# encode person ids in transcript
transcript['person_id'] = transcript['person_id'].apply(lambda x: person_id_dict[x])

# check results
transcript.sample()

,event,person_id,time,amount,offer_id
221677,offer viewed,p_15877,510,NaN,2906b810c7d4411798c6938adc9daaa5


In [22]:
# additional safety check
assert transcript.loc[271980, 'person_id'] == person_id_dict['98b81a8ffdd14bb986d80673491171a7']
print(transcript.loc[271980, :])

event        transaction
person_id         p_1252
time                 600
amount                12
offer_id             NaN
Name: 271980, dtype: object


In [23]:
# encode offer ids
portfolio = portfolio.sort_values('offer_type')
encoded_offer_ids, offer_id_dict = map_id(portfolio, 'offer_id', prefix="o_")
portfolio['offer_id'] = encoded_offer_ids

# check results
portfolio

,difficulty,duration,offer_id,offer_type,reward,web,email,mobile,social,prop_rewards,rel_difficulty,duration_hours
0,10,7,o_1,bogo,10,0.0,1.0,1.0,1.0,1.000000,1.428571,168
1,10,5,o_2,bogo,10,1.0,1.0,1.0,1.0,1.000000,2.000000,120
3,5,7,o_3,bogo,5,1.0,1.0,1.0,0.0,1.000000,0.714286,168
8,5,5,o_4,bogo,5,1.0,1.0,1.0,1.0,1.000000,1.000000,120
4,20,10,o_5,discount,5,1.0,1.0,0.0,0.0,0.250000,2.000000,240
5,7,7,o_6,discount,3,1.0,1.0,1.0,1.0,0.428571,1.000000,168
6,10,10,o_7,discount,2,1.0,1.0,1.0,1.0,0.200000,1.000000,240
9,10,7,o_8,discount,2,1.0,1.0,1.0,0.0,0.200000,1.428571,168
2,0,4,o_9,informational,0,1.0,1.0,1.0,0.0,NaN,0.000000,96
7,0,3,o_10,informational,0,0.0,1.0,1.0,1.0,NaN,0.000000,72


In [24]:
transcript.head()

,event,person_id,time,amount,offer_id
0,offer received,p_4,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,offer received,p_5,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,offer received,p_6,0,NaN,2906b810c7d4411798c6938adc9daaa5
3,offer received,p_7,0,NaN,fafdcd668e3743c1bb461111dcafc2a4
4,offer received,p_8,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0


In [25]:
# encode offer ids in transcript - different treatment because of NaN

new_ids = []
for row in transcript['offer_id']:
    if row in offer_id_dict.keys():
        id = offer_id_dict[row]
    else:
        id = np.NaN
        
    new_ids.append(id)
    
transcript['offer_id'] = new_ids

# check results
transcript.sample(5)

,event,person_id,time,amount,offer_id
147740,transaction,p_5319,396,19.22,NaN
289251,transaction,p_12681,642,7.14,NaN
129250,transaction,p_11956,342,6.5,NaN
228390,offer viewed,p_3507,528,NaN,o_7
130561,transaction,p_2495,348,22.36,NaN


In [28]:
# additional safety checks
assert transcript.loc[228390, 'offer_id'] == offer_id_dict['fafdcd668e3743c1bb461111dcafc2a4']
assert transcript.loc[142891, 'offer_id'] == offer_id_dict['4d5c57ea9a6940dd891ad53e9dbe8da0']
print(transcript.loc[142891, :])

event        offer viewed
person_id         p_11287
time                  378
amount                NaN
offer_id              o_2
Name: 142891, dtype: object


## Look at cleaned files, store for later use

In [31]:
portfolio

,difficulty,duration,offer_id,offer_type,reward,web,email,mobile,social,prop_rewards,rel_difficulty,duration_hours
0,10,7,o_1,bogo,10,0.0,1.0,1.0,1.0,1.000000,1.428571,168
1,10,5,o_2,bogo,10,1.0,1.0,1.0,1.0,1.000000,2.000000,120
3,5,7,o_3,bogo,5,1.0,1.0,1.0,0.0,1.000000,0.714286,168
8,5,5,o_4,bogo,5,1.0,1.0,1.0,1.0,1.000000,1.000000,120
4,20,10,o_5,discount,5,1.0,1.0,0.0,0.0,0.250000,2.000000,240
5,7,7,o_6,discount,3,1.0,1.0,1.0,1.0,0.428571,1.000000,168
6,10,10,o_7,discount,2,1.0,1.0,1.0,1.0,0.200000,1.000000,240
9,10,7,o_8,discount,2,1.0,1.0,1.0,0.0,0.200000,1.428571,168
2,0,4,o_9,informational,0,1.0,1.0,1.0,0.0,NaN,0.000000,96
7,0,3,o_10,informational,0,0.0,1.0,1.0,1.0,NaN,0.000000,72


In [32]:
profile.head()

,age,gender,person_id,income,member_since
0,118,None,p_1,NaN,529
1,55,F,p_2,112000.0,376
2,118,None,p_3,NaN,14
3,75,F,p_4,100000.0,443
4,118,None,p_5,NaN,356


In [33]:
transcript.head()

,event,person_id,time,amount,offer_id
0,offer received,p_4,0,NaN,o_3
1,offer received,p_5,0,NaN,o_5
2,offer received,p_6,0,NaN,o_8
3,offer received,p_7,0,NaN,o_7
4,offer received,p_8,0,NaN,o_2


In [35]:
# safe clean data

portfolio_clean = Path.cwd() / "data" / "processed" / "portfolio_clean.csv"
profile_clean = Path.cwd() / "data" / "processed" / "profile_clean.csv"
transcript_clean = Path.cwd() / "data" / "processed" / "transcript_clean.csv"

portfolio.to_csv(portfolio_clean, index=False)
profile.to_csv(profile_clean, index=False)
transcript.to_csv(transcript_clean, index=False)

---